In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,2021-01-31T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1325,204,1529,40760,...,NaN,8.0,NaN,NaN,136103.0,0.0,1506448.0,278011.0,ITG,ITG2
7199,2021-01-31T17:00:00,ITA,9,Toscana,43.769231,11.255889,648,96,744,8431,...,NaN,8.0,NaN,NaN,134360.0,96.0,2151364.0,85654.0,ITG,ITG1
7200,2021-01-31T17:00:00,ITA,10,Umbria,43.106758,12.388247,361,54,415,5214,...,Si fa presente come un ricovero (in terapia in...,2.0,NaN,NaN,36051.0,0.0,595305.0,33835.0,ITI,ITI2
7201,2021-01-31T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,32,3,35,198,...,NaN,0.0,NaN,NaN,7797.0,0.0,69198.0,941.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,2021-01-31T17:00:00,19,Sicilia,1325,204,1529,40760,42289,-579,716,...,NaN,NaN,136103,1784459,985615.0,8.0,136103.0,0.0,1506448.0,278011.0
7199,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,510,...,NaN,NaN,134456,2237018,1195439.0,8.0,134360.0,96.0,2151364.0,85654.0
7200,2021-01-31T17:00:00,10,Umbria,361,54,415,5214,5629,118,294,...,NaN,NaN,36051,629140,283677.0,2.0,36051.0,0.0,595305.0,33835.0
7201,2021-01-31T17:00:00,2,Valle d'Aosta,32,3,35,198,233,-15,16,...,NaN,NaN,7797,70139,41505.0,0.0,7797.0,0.0,69198.0,941.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-31', '2021-01-30')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-31T17:00:00,1,Piemonte,2122,146,2268,10044,12312,-70,538,...,NaN,NaN,223097,2072804,1099174.0,0.0,221177.0,1920.0,1882944.0,189860.0
1,2021-01-31T17:00:00,2,Valle d'Aosta,32,3,35,198,233,-15,16,...,NaN,NaN,7797,70139,41505.0,0.0,7797.0,0.0,69198.0,941.0
2,2021-01-31T17:00:00,3,Lombardia,3503,362,3865,47212,51077,936,1438,...,NaN,NaN,538054,5665413,2837007.0,7.0,536665.0,1389.0,5531042.0,134371.0
3,2021-01-31T17:00:00,5,Veneto,1430,217,1647,32298,33945,-732,499,...,NaN,NaN,312185,4153641,1323060.0,5.0,310868.0,1317.0,3751417.0,402224.0
4,2021-01-31T17:00:00,6,Friuli Venezia Giulia,590,60,650,10683,11333,39,396,...,NaN,NaN,67534,1146864,416815.0,8.0,65047.0,2487.0,1111452.0,35412.0
5,2021-01-31T17:00:00,7,Liguria,612,65,677,3759,4436,41,307,...,NaN,NaN,69668,862130,377791.0,3.0,69668.0,0.0,820447.0,41683.0
6,2021-01-31T17:00:00,8,Emilia-Romagna,2112,206,2318,44550,46868,509,1277,...,NaN,NaN,218651,3103697,1357767.0,10.0,218595.0,56.0,2986157.0,117540.0
7,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,510,...,NaN,NaN,134456,2237018,1195439.0,8.0,134360.0,96.0,2151364.0,85654.0
8,2021-01-31T17:00:00,10,Umbria,361,54,415,5214,5629,118,294,...,NaN,NaN,36051,629140,283677.0,2.0,36051.0,0.0,595305.0,33835.0
9,2021-01-31T17:00:00,11,Marche,529,72,601,8657,9258,98,362,...,NaN,NaN,55487,690203,415370.0,4.0,55487.0,0.0,664718.0,25485.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-30T17:00:00,1,Piemonte,2127,150,2277,10105,12382,-155,727,...,NaN,NaN,222559,2063546,1095712.0,14.0,220686.0,1873.0,1878337.0,185209.0
1,2021-01-30T17:00:00,2,Valle d'Aosta,35,4,39,209,248,-5,1,...,NaN,NaN,7781,69831,41402.0,1.0,7781.0,0.0,68965.0,866.0
2,2021-01-30T17:00:00,3,Lombardia,3454,377,3831,46310,50141,627,1832,...,NaN,NaN,536616,5640919,2829643.0,19.0,535301.0,1315.0,5510184.0,130735.0
3,2021-01-30T17:00:00,5,Veneto,1438,222,1660,33017,34677,-2288,792,...,NaN,NaN,311686,4132597,1320747.0,16.0,310348.0,1338.0,3741618.0,390979.0
4,2021-01-30T17:00:00,6,Friuli Venezia Giulia,596,57,653,10641,11294,-77,415,...,NaN,NaN,67138,1142473,415009.0,3.0,64737.0,2401.0,1107990.0,34483.0
5,2021-01-30T17:00:00,7,Liguria,588,62,650,3745,4395,90,255,...,NaN,NaN,69361,855992,375966.0,0.0,69361.0,0.0,817538.0,38454.0
6,2021-01-30T17:00:00,8,Emilia-Romagna,2124,203,2327,44032,46359,-986,1314,...,NaN,NaN,217377,3090178,1354289.0,15.0,217323.0,54.0,2978096.0,112082.0
7,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,603,...,NaN,NaN,133946,2223132,1188245.0,1.0,133853.0,93.0,2141135.0,81997.0
8,2021-01-30T17:00:00,10,Umbria,346,55,401,5110,5511,34,228,...,NaN,NaN,35757,623158,282438.0,6.0,35757.0,0.0,591529.0,31629.0
9,2021-01-30T17:00:00,11,Marche,542,71,613,8547,9160,153,465,...,NaN,NaN,55125,685976,412859.0,3.0,55125.0,0.0,661459.0,24517.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-31T17:00:00,1,Piemonte,2122,146,2268,10044,12312,-70,538,...,NaN,NaN,223097,2072804,1099174.0,0.0,221177.0,1920.0,1882944.0,189860.0
1,2021-01-31T17:00:00,2,Valle d'Aosta,32,3,35,198,233,-15,16,...,NaN,NaN,7797,70139,41505.0,0.0,7797.0,0.0,69198.0,941.0
2,2021-01-31T17:00:00,3,Lombardia,3503,362,3865,47212,51077,936,1438,...,NaN,NaN,538054,5665413,2837007.0,7.0,536665.0,1389.0,5531042.0,134371.0
3,2021-01-31T17:00:00,4,Trentino-Alto Adige,401,67,468,6619,7087,-10709,602,...,0.0,0.0,67694,1026221,341868.0,1.0,61496.0,6198.0,936184.0,90037.0
4,2021-01-31T17:00:00,5,Veneto,1430,217,1647,32298,33945,-732,499,...,NaN,NaN,312185,4153641,1323060.0,5.0,310868.0,1317.0,3751417.0,402224.0
5,2021-01-31T17:00:00,6,Friuli Venezia Giulia,590,60,650,10683,11333,39,396,...,NaN,NaN,67534,1146864,416815.0,8.0,65047.0,2487.0,1111452.0,35412.0
6,2021-01-31T17:00:00,7,Liguria,612,65,677,3759,4436,41,307,...,NaN,NaN,69668,862130,377791.0,3.0,69668.0,0.0,820447.0,41683.0
7,2021-01-31T17:00:00,8,Emilia-Romagna,2112,206,2318,44550,46868,509,1277,...,NaN,NaN,218651,3103697,1357767.0,10.0,218595.0,56.0,2986157.0,117540.0
8,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,510,...,NaN,NaN,134456,2237018,1195439.0,8.0,134360.0,96.0,2151364.0,85654.0
9,2021-01-31T17:00:00,10,Umbria,361,54,415,5214,5629,118,294,...,NaN,NaN,36051,629140,283677.0,2.0,36051.0,0.0,595305.0,33835.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-01-30T17:00:00,1,Piemonte,2127,150,2277,10105,12382,-155,...,NaN,NaN,222559,2063546,1095712.0,14.0,220686.0,1873.0,1878337.0,185209.0
1,1,2021-01-30T17:00:00,2,Valle d'Aosta,35,4,39,209,248,-5,...,NaN,NaN,7781,69831,41402.0,1.0,7781.0,0.0,68965.0,866.0
2,2,2021-01-30T17:00:00,3,Lombardia,3454,377,3831,46310,50141,627,...,NaN,NaN,536616,5640919,2829643.0,19.0,535301.0,1315.0,5510184.0,130735.0
3,19,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,...,0.0,0.0,67092,1018897,340192.0,3.0,61127.0,5965.0,931630.0,87267.0
4,3,2021-01-30T17:00:00,5,Veneto,1438,222,1660,33017,34677,-2288,...,NaN,NaN,311686,4132597,1320747.0,16.0,310348.0,1338.0,3741618.0,390979.0
5,4,2021-01-30T17:00:00,6,Friuli Venezia Giulia,596,57,653,10641,11294,-77,...,NaN,NaN,67138,1142473,415009.0,3.0,64737.0,2401.0,1107990.0,34483.0
6,5,2021-01-30T17:00:00,7,Liguria,588,62,650,3745,4395,90,...,NaN,NaN,69361,855992,375966.0,0.0,69361.0,0.0,817538.0,38454.0
7,6,2021-01-30T17:00:00,8,Emilia-Romagna,2124,203,2327,44032,46359,-986,...,NaN,NaN,217377,3090178,1354289.0,15.0,217323.0,54.0,2978096.0,112082.0
8,7,2021-01-30T17:00:00,9,Toscana,648,95,743,8340,9083,81,...,NaN,NaN,133946,2223132,1188245.0,1.0,133853.0,93.0,2141135.0,81997.0
9,8,2021-01-30T17:00:00,10,Umbria,346,55,401,5110,5511,34,...,NaN,NaN,35757,623158,282438.0,6.0,35757.0,0.0,591529.0,31629.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-01-31T17:00:00,1,Piemonte,2122,146,2268,10044,12312,-70,538,...,NaN,223097,2072804,1099174.0,0.0,221177.0,1920.0,1882944.0,189860.0,4341375
1,2021-01-31T17:00:00,2,Valle d'Aosta,32,3,35,198,233,-15,16,...,NaN,7797,70139,41505.0,0.0,7797.0,0.0,69198.0,941.0,125501
2,2021-01-31T17:00:00,3,Lombardia,3503,362,3865,47212,51077,936,1438,...,NaN,538054,5665413,2837007.0,7.0,536665.0,1389.0,5531042.0,134371.0,10103969
3,2021-01-31T17:00:00,4,Trentino-Alto Adige,401,67,468,6619,7087,-10709,602,...,0.0,67694,1026221,341868.0,1.0,61496.0,6198.0,936184.0,90037.0,1074819
4,2021-01-31T17:00:00,5,Veneto,1430,217,1647,32298,33945,-732,499,...,NaN,312185,4153641,1323060.0,5.0,310868.0,1317.0,3751417.0,402224.0,4907704
5,2021-01-31T17:00:00,6,Friuli Venezia Giulia,590,60,650,10683,11333,39,396,...,NaN,67534,1146864,416815.0,8.0,65047.0,2487.0,1111452.0,35412.0,1211357
6,2021-01-31T17:00:00,7,Liguria,612,65,677,3759,4436,41,307,...,NaN,69668,862130,377791.0,3.0,69668.0,0.0,820447.0,41683.0,1543127
7,2021-01-31T17:00:00,8,Emilia-Romagna,2112,206,2318,44550,46868,509,1277,...,NaN,218651,3103697,1357767.0,10.0,218595.0,56.0,2986157.0,117540.0,4467118
8,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,510,...,NaN,134456,2237018,1195439.0,8.0,134360.0,96.0,2151364.0,85654.0,3722729
9,2021-01-31T17:00:00,10,Umbria,361,54,415,5214,5629,118,294,...,NaN,36051,629140,283677.0,2.0,36051.0,0.0,595305.0,33835.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-01-31T17:00:00,1,Piemonte,2122,146,2268,10044,12312,-70,538,...,223097,2072804,1099174.0,0.0,221177.0,1920.0,1882944.0,189860.0,4341375,575
1,2021-01-31T17:00:00,2,Valle d'Aosta,32,3,35,198,233,-15,16,...,7797,70139,41505.0,0.0,7797.0,0.0,69198.0,941.0,125501,20
2,2021-01-31T17:00:00,3,Lombardia,3503,362,3865,47212,51077,936,1438,...,538054,5665413,2837007.0,7.0,536665.0,1389.0,5531042.0,134371.0,10103969,1036
3,2021-01-31T17:00:00,4,Trentino-Alto Adige,401,67,468,6619,7087,-10709,602,...,67694,1026221,341868.0,1.0,61496.0,6198.0,936184.0,90037.0,1074819,106
4,2021-01-31T17:00:00,5,Veneto,1430,217,1647,32298,33945,-732,499,...,312185,4153641,1323060.0,5.0,310868.0,1317.0,3751417.0,402224.0,4907704,1016
5,2021-01-31T17:00:00,6,Friuli Venezia Giulia,590,60,650,10683,11333,39,396,...,67534,1146864,416815.0,8.0,65047.0,2487.0,1111452.0,35412.0,1211357,180
6,2021-01-31T17:00:00,7,Liguria,612,65,677,3759,4436,41,307,...,69668,862130,377791.0,3.0,69668.0,0.0,820447.0,41683.0,1543127,209
7,2021-01-31T17:00:00,8,Emilia-Romagna,2112,206,2318,44550,46868,509,1277,...,218651,3103697,1357767.0,10.0,218595.0,56.0,2986157.0,117540.0,4467118,634
8,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,510,...,134456,2237018,1195439.0,8.0,134360.0,96.0,2151364.0,85654.0,3722729,523
9,2021-01-31T17:00:00,10,Umbria,361,54,415,5214,5629,118,294,...,36051,629140,283677.0,2.0,36051.0,0.0,595305.0,33835.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-31T17:00:00,3,Lombardia,3503,362,3865,47212,51077,936,1438,...,34.94,902,-394,24494,7364.0,24,478,5.87,0.01423,5.33
1,2021-01-31T17:00:00,15,Campania,1417,99,1516,60906,62422,766,1401,...,19.60,777,35,17094,12252.0,10,625,8.20,0.02421,3.84
2,2021-01-31T17:00:00,8,Emilia-Romagna,2112,206,2318,44550,46868,509,1277,...,32.49,518,-37,13519,3478.0,22,743,9.45,0.02859,4.89
3,2021-01-31T17:00:00,16,Puglia,1397,164,1561,51183,52744,587,1069,...,44.44,578,198,8186,1078.0,13,469,13.06,0.02667,3.07
4,2021-01-31T17:00:00,12,Lazio,2388,284,2672,60529,63201,-594,943,...,33.53,-577,-195,23198,15383.0,39,1498,4.07,0.01608,3.50
5,2021-01-31T17:00:00,19,Sicilia,1325,204,1529,40760,42289,-579,716,...,34.69,-555,-130,32850,4950.0,35,1260,2.18,0.01441,2.74
6,2021-01-31T17:00:00,4,Trentino-Alto Adige,401,67,468,6619,7087,-10709,602,...,63.21,-10721,-268,7324,1676.0,5,11306,8.22,0.05601,6.30
7,2021-01-31T17:00:00,1,Piemonte,2122,146,2268,10044,12312,-70,538,...,25.39,-61,-189,9258,3462.0,14,594,5.81,0.01239,5.14
8,2021-01-31T17:00:00,9,Toscana,648,96,744,8431,9175,92,510,...,18.36,91,-93,13886,7194.0,8,410,3.67,0.01370,3.61
9,2021-01-31T17:00:00,5,Veneto,1430,217,1647,32298,33945,-732,499,...,21.36,-719,-293,21044,2313.0,9,1222,2.37,0.01017,6.36


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,67,39,106,63.21
1,Marche,72,71,143,50.35
2,Umbria,54,63,117,46.15
3,Puglia,164,205,369,44.44
4,Lombardia,362,674,1036,34.94
5,Sicilia,204,384,588,34.69
6,Lazio,284,563,847,33.53
7,Friuli Venezia Giulia,60,120,180,33.33
8,Emilia-Romagna,206,428,634,32.49
9,Liguria,65,144,209,31.10


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,2021-01-29T17:00:00,22,P.A. Trento,187,34,221,2150,2371,208,307,...,NaN,NaN,27162,523153,161081.0,1.0,25377.0,1785.0,507730.0,15423.0
682,2021-01-30T17:00:00,21,P.A. Bolzano,216,32,248,15083,15331,469,640,...,NaN,NaN,39700,492413,178834.0,2.0,35711.0,3989.0,422081.0,70332.0
683,2021-01-30T17:00:00,22,P.A. Trento,173,35,208,2257,2465,94,230,...,NaN,NaN,27392,526484,161358.0,1.0,25416.0,1976.0,509549.0,16935.0
684,2021-01-31T17:00:00,21,P.A. Bolzano,223,32,255,4363,4618,-10713,467,...,NaN,NaN,40167,497235,180213.0,0.0,36043.0,4124.0,425065.0,72170.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,2021-01-27T17:00:00,4,Trentino-Alto Adige,418,67,485,15827,16312,287,790,...,4.0,11806.0,1053.0,309.0,13.81,2.97,6.69,75.02,1.52,1.79
339,2021-01-28T17:00:00,4,Trentino-Alto Adige,407,64,471,16232,16703,391,769,...,4.0,11509.0,1022.0,405.0,13.59,2.82,6.68,75.24,-4.48,2.40
340,2021-01-29T17:00:00,4,Trentino-Alto Adige,406,64,470,16763,17233,530,897,...,4.0,11105.0,858.0,531.0,13.62,2.73,8.08,104.55,0.00,3.17
341,2021-01-30T17:00:00,4,Trentino-Alto Adige,389,67,456,17340,17796,563,870,...,9.0,10953.0,1022.0,577.0,14.69,2.56,7.94,85.13,4.69,3.27


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
